In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
from torch import nn
import torch.optim as optim
from torchvision import datasets, transforms

from model import CNN
from data_loader import ImageTransform, ImageDataset

In [2]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_train.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity
0,0007de18844b0dbbb5e1f607da0606e0,0,1,1,1,0,0,1,0,0,0,0,0,63
1,0009c66b9439883ba2750fb825e1d7db,0,1,1,0,0,0,0,0,0,0,0,0,42
2,0013fd999caf9a3efe1352ca1b0d937e,0,1,1,1,0,0,0,0,1,1,0,0,28
3,0018df346ac9c1d8413cfcc888ca8246,0,1,1,1,0,0,0,0,0,0,0,0,15
4,001dc955e10590d3ca4673f034feeef2,0,0,0,1,0,0,1,0,0,0,0,0,72


In [3]:
df_train["img_path"] = df_train["Id"].apply(lambda x: f"train/{x}.jpg")
X_train = df_train.drop(["Id", "Pawpularity"], axis=1)
y_train = df_train["Pawpularity"] / 100
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
X_train = X_train.reset_index(drop=True)
X_val = X_val.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_val = y_val.reset_index(drop=True)

In [4]:
EPOCH_NUM = 10
BATCH_SIZE = 128
mean, std = 0, 1
size = (256, 256)
train = ImageDataset(data_list=X_train["img_path"], transform=ImageTransform(mean, std, size), label_list=y_train)
train_dataloader = torch.utils.data.DataLoader(train, BATCH_SIZE, shuffle=True)
val = ImageDataset(data_list=X_val["img_path"], transform=ImageTransform(mean, std, size), label_list=y_val)
val_dataloader = torch.utils.data.DataLoader(val, BATCH_SIZE, shuffle=True)

In [5]:
in_channels = 3
hidden_channels = 32
out_channels = 1
classifier = CNN(in_channels, out_channels, hidden_channels)
criterion = nn.MSELoss()
optimizer = optim.Adam(classifier.parameters())

In [7]:
print("Train")
st = datetime.datetime.now()
for epoch in range(EPOCH_NUM):
    total_loss = 0
    for images, labels in train_dataloader:
        images = images.to(torch.float)
        labels = labels.to(torch.float).view(-1)
        optimizer.zero_grad()
        outputs = classifier.forward(images)
        loss = criterion(outputs, labels).to(torch.float)
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    if (epoch+1) % 1 == 0:
        print(f"epoch:{epoch+1} total loss:{total_loss}")

    total_loss = 0
    correct = 0
    total = 0
    classifier.eval()
    with torch.no_grad():
        for images, labels in val_dataloader:
            images = images.to(torch.float)
            labels = labels.to(torch.float).view(-1)
            outputs = classifier(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total += labels.size(0)
    val_loss = total_loss / total
    if (epoch+1) % 1 == 0:
        ed = datetime.datetime.now()
        print(f"val loss:{val_loss} time:{ed-st}")
        print("#"*30)
        st = datetime.datetime.now()

Train


C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch:1 total loss:1657.544319972396


C:\Users\ryuhe\Anaconda3\envs\KAGGLE\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([46])) that is different to the input size (torch.Size([46, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


val acc:0.0 time:0:18:04.835010
####################
epoch:2 total loss:13.133052259683609
val acc:0.0 time:0:15:30.302980
####################
epoch:3 total loss:12.700983628630638
val acc:0.0 time:0:13:27.561399
####################
epoch:4 total loss:12.42160576581955
val acc:0.0 time:0:10:42.909000
####################
epoch:5 total loss:11.955485314130783
val acc:0.0 time:0:10:27.314213
####################
epoch:6 total loss:11.672575026750565
val acc:0.0 time:0:10:35.874004
####################
epoch:7 total loss:11.483851313591003
val acc:0.0 time:0:10:02.347459
####################
epoch:8 total loss:11.089772641658783
val acc:0.0 time:0:09:52.776368
####################
epoch:9 total loss:10.845300853252411
val acc:0.0 time:0:09:51.104039
####################
epoch:10 total loss:10.463958829641342
val acc:0.0 time:0:09:53.779255
####################
